In [1]:
import warnings
warnings.filterwarnings('ignore')
import os
import re
import json
import string
import numpy as np
import tensorflow as tf
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from transformers import AutoTokenizer, TFAutoModel
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

from modules.utils import *
from modules.dataframe_tools import *
from keras.models import load_model

In [2]:
def print_observation(record):
    """     id = record["id"]
        question = record["question"]
        pred = record["pred"]
        true_answer = record["true_answer"]
        context = record["context"]
        
        print("id:\t\t{}n".format(id))
        print("Question: \t{}".format(question))
        print("Paragraph:\t{}".format(context))
        print("True Answer:\t{}".format(true_answer))
        print("predicted:\t{}".format(pred))
    """   
    for column in record.index.to_list():
        print("{:20s}{}".format(column + ":", record[column]))
    print("==============================================================\n\n")



In [3]:
path_to_json = "dataset/training_set.json"
df_orig = create_df_test(path_to_json, [], parag_lift=2)
df = df_orig.sample(n=15)
'''
path_to_pred = "./output_ensemble.json"
with open(path_to_pred) as f:
    json_txt = json.load(f)
'''


'\npath_to_pred = "./output_ensemble.json"\nwith open(path_to_pred) as f:\n    json_txt = json.load(f)\n'

In [4]:
names = [name for name in os.listdir() if name.startswith("output_")]
predictions ={}
for name in names:
    with open("./"+ name, "r") as file:
        txt = json.load(file)
    pred = [ txt.get(k) for k in df["id"]]
    predictions[name[:-5]] = pred

In [5]:
df.columns

Index(['id', 'title', 'context', 'question', 'answer_text', 'start_idx'], dtype='object')

In [6]:
df3 = {"id": df["id"],
        "context": df["context"].tolist(),
        "question": df["question"].tolist(),
        "true_answer": df["answer_text"].tolist()
        }
df3.update(predictions)

df3 = pd.DataFrame.from_dict(df3, orient ="columns")

In [7]:
df3

,id,context,question,true_answer,output_6Mod,output_8Mod,output_CNN,output_Multiply,output_Vanilla,output_VC,output_VCM
110,57338a51d058e614000b5cf2,Holy Cross Father John Francis O'Hara was elec...,Irvin Abell was given what award by Notre Dame?,Laetare Medal,Laetare Medal,Laetare Medal,Laetare Medal,Laetare Medal,Laetare Medal,Laetare Medal,Laetare Medal
175,5733a55a4776f41900660f3e,The School of Architecture was established in ...,Which prestigious prize does the School of Arc...,Driehaus Architecture Prize,Driehaus Architecture Prize,Driehaus Architecture Prize,Driehaus Architecture Prize,Driehaus Architecture Prize,Driehaus Architecture Prize,Driehaus Architecture Prize,Driehaus Architecture Prize
415,56bf91c6a10cfb140055117f,"In November 2003, she embarked on the Dangerou...",In what year did Beyonce embark on her Dangero...,November 2003,2003,2003,2003,2003,2003,2003,2003
224,5733c29c4776f419006611b8,Notre Dame's conference affiliations for all o...,To what conference did the Fighting Irish go a...,the ACC,ACC,ACC,ACC,the ACC,ACC,ACC,ACC
633,56d4d2232ccc5a1400d83266,"On January 7, 2012, Beyoncé gave birth to a da...",How was Blue Ivy credited on Glory?,B.I.C.,B.I.C.,B.I.C.,B.I.C.,B.I.C.,B.I.C.,B.I.C.,B.I.C.
330,56bf76ef3aeaaa14008c9665,Beyoncé attended St. Mary's Elementary School ...,I which church was Beyonce a member and soloi...,St. John's United Methodist Church,St. John's United Methodist Church,St. John's United Methodist Church,St. John's United Methodist Church,St. John's United Methodist Church,St. John's United Methodist Church,St. John's United Methodist Church,St. John's United Methodist Church
954,56d4f5d32ccc5a1400d83381,"In October 2014, it was announced that Beyoncé...",Who did Beyoncé and Parkwood Entertainment par...,Topshop,Topshop,Topshop,Topshop,Topshop,Topshop,Topshop,Topshop
259,5733ccbe4776f41900661270,"In the film Knute Rockne, All American, Knute ...","Ronald Reagan had a nickname, what was it?",The Gipper,The Gipper,The Gipper,The Gipper,The Gipper,The Gipper,The Gipper,The Gipper
898,56d4eb762ccc5a1400d83352,Beyoncé has received numerous awards. As a sol...,When did Beyoncé receive the Legend Award?,the 2008 World Music Awards,2008 World Music Awards,2008 World Music Awards,2008,2008 World Music Awards,2008 World Music Awards,2008,2008 World Music Awards
723,56bfbacaa10cfb1400551287,"In 2001, she became the first African-American...",Where does she place in writing credits for t...,third woman,third,third,,third,third,third,third


In [8]:

for id, row in df3.iterrows():
    print(row.index.to_list())
    
    print_observation(row)

['id', 'context', 'question', 'true_answer', 'output_6Mod', 'output_8Mod', 'output_CNN', 'output_Multiply', 'output_Vanilla', 'output_VC', 'output_VCM']
id:                 57338a51d058e614000b5cf2
context:            Holy Cross Father John Francis O'Hara was elected vice-president in 1933 and president of Notre Dame in 1934. During his tenure at Notre Dame, he brought numerous refugee intellectuals to campus; he selected Frank H. Spearman, Jeremiah D. M. Ford, Irvin Abell, and Josephine Brownson for the Laetare Medal, instituted in 1883. O'Hara strongly believed that the Fighting Irish football team could be an effective means to "acquaint the public with the ideals that dominate" Notre Dame. He wrote, "Notre Dame football is a spiritual service because it is played for the honor and glory of God and of his Blessed Mother. When St. Paul said: 'Whether you eat or drink, or whatsoever else you do, do all for the glory of God,' he included football."
question:           Irvin Abell was g